In [28]:
import pandas as pd

In [2]:
import h5py
with h5py.File('data/train.h5', 'r') as f:
    print(list(f.keys()))  # Verifica los nombres de los datasets


['x', 'y']


In [4]:
import h5py

with h5py.File('data/train.h5', 'r') as f:
    data = f['x'][:]  # Series de tiempo (ya no se llama 'data', sino 'x')
    labels = f['y'][:]  # Etiquetas (ya no se llama 'label', sino 'y')

# Verifica las formas:
print("Forma de los datos (x):", data.shape)  # Debería ser [n_muestras, 64_electrodos, 256_puntos]
print("Forma de las etiquetas (y):", labels.shape)  # Debería ser [n_muestras,]

Forma de los datos (x): (204, 1, 18530)
Forma de las etiquetas (y): (204,)


In [32]:
data.shape

(204, 1, 18530)

In [47]:
# Aplanar la señal (quita la dimensión "1")
X_flat = data.reshape(data.shape[0], data.shape[2])

# Crear DataFrame
df = pd.DataFrame(X_flat)
#df["label"] = labels  # añadimos la etiqueta al final

In [48]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,18520,18521,18522,18523,18524,18525,18526,18527,18528,18529
0,-0.005859,-0.011414,-0.010773,-0.010559,-0.008850,-0.008636,-0.009094,-0.008850,-0.009369,-0.009888,...,0.012573,0.007721,0.002075,-0.003174,-0.007843,-0.012573,-0.017670,-0.019897,-0.021637,-0.018860
1,0.002380,0.005127,0.005737,0.007263,0.008575,0.010742,0.012970,0.017212,0.022003,0.027954,...,0.003815,0.005707,0.006683,0.006012,0.003601,0.000427,-0.001984,-0.003998,-0.005981,-0.007660
2,0.058075,0.122040,0.108307,0.113769,0.105316,0.103912,0.094849,0.091187,0.080170,0.074432,...,-0.009613,-0.008301,-0.005402,-0.007690,-0.006592,-0.007904,-0.009674,-0.013153,-0.014130,-0.018250
3,-0.007385,-0.011932,-0.004700,-0.003143,0.005493,0.006744,0.012085,0.012909,0.018097,0.021606,...,-0.003082,-0.007050,-0.010956,-0.013916,-0.017944,-0.019867,-0.025604,-0.027161,-0.030273,-0.039368
4,-0.000458,-0.001099,0.003723,0.002930,0.004059,0.003906,0.003265,0.002777,0.000488,-0.001221,...,0.040039,0.040680,0.041901,0.040710,0.038727,0.025818,0.019470,0.010010,0.004242,0.003235


In [49]:
# Cantidad de nulos en el dataframe
print(df.isnull().sum().sum())

0


In [50]:
import numpy as np
from tsfresh import extract_features

# melt convierte tu DataFrame de ancho a largo
df_long = pd.melt(df.reset_index(), id_vars=['index'], var_name="time", value_name="value")

# renombramos columnas para TsFresh
df_long = df_long.rename(columns={'index':'id'})

# id = cuál muestra pertenece cada valor
df_long["id"] = df_long["id"].astype(int)
df_long["time"] = df_long["time"].astype(int)

In [52]:
features = extract_features(df_long, column_id="id", column_sort="time", disable_progressbar=True)

MemoryError: Unable to allocate 5.12 GiB for an array with shape (18529, 18529, 2) and data type float64

In [ ]:
features.

In [9]:
# Paso 1: Cargar datos (listo)


# Paso 2: EDA
import numpy as np
print("Número de muestras por clase:", np.unique(labels, return_counts=True))

# Paso 3: Preparar DataFrame para TsFresh
import pandas as pd
df_list = []
for i in range(data.shape[0]):
    for t in range(data.shape[2]):
        df_list.append({'id': i, 'time': t, 'value': data[i,0,t]})
df_tsfresh = pd.DataFrame(df_list)

# Paso 4: Extraer características (ejecutar en una máquina potente o submuestrear)
from tsfresh import extract_features
features = extract_features(df_tsfresh, column_id='id', column_sort='time')

Número de muestras por clase: (array([0, 1]), array([149,  55]))


Feature Extraction:   0%|          | 0/19 [01:02<?, ?it/s]


MemoryError: Unable to allocate 5.12 GiB for an array with shape (18529, 18529, 2) and data type float64